In [2]:
'''
    data loader -> ./model/data_loader.py
'''

import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')


In [4]:
'''
    examine data

    note: if executing in jupyter notebook, using imshow() after loading dataset could result in memory overflow and kernel restart
'''

import matplotlib.pyplot as plt
import numpy as np

def img_show(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()

img_show(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [3]:
'''
    define a network -> ./model/net.py
'''

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module) :
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()

In [4]:
'''
    define loss and optimizer 
'''

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
'''
    main training loop -> ./train.py
'''

for epoch in range(2):

    running_loss = 0

    for i, data in enumerate(trainloader,0):
        
        inputs, labels = data
        
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')
modelPATH = './trained_models/cifar_net.pth'
torch.save(net.state_dict(),modelPATH)

[1,  2000] loss: 2.283
[1,  4000] loss: 2.013
[1,  6000] loss: 1.754
[1,  8000] loss: 1.597
[1, 10000] loss: 1.521
[1, 12000] loss: 1.466
[2,  2000] loss: 1.393
[2,  4000] loss: 1.359
[2,  6000] loss: 1.345
[2,  8000] loss: 1.320
[2, 10000] loss: 1.299
[2, 12000] loss: 1.259
Finished Training


In [5]:
'''
    test -> ./test.py
'''

dataiter = iter(testloader)
images, labels = dataiter.next()

#imshow(torchvision.utils.make_grid(images))
print('GroundTruth:', ' '.join('%5s' % classes[labels[j]] for j in range(4)))


net = Net()
net.load_state_dict(torch.load(modelPATH))

outputs = net(images)
_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

GroundTruth:   cat  ship  ship plane
Predicted:    cat  ship  ship  ship


In [6]:
'''
    accuracy on full test set
'''

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10,000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10,000 test images: 54 %


In [7]:
'''
    accuracy by class label
'''

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 52 %
Accuracy of   car : 63 %
Accuracy of  bird : 48 %
Accuracy of   cat : 37 %
Accuracy of  deer : 40 %
Accuracy of   dog : 58 %
Accuracy of  frog : 58 %
Accuracy of horse : 54 %
Accuracy of  ship : 71 %
Accuracy of truck : 64 %
